In [38]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib3 
http = urllib3.PoolManager()
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader as web
import glob
import os
import math
pd.set_option('display.max_rows', 500)

In [39]:
def get_ROIC(df, years_average1 = 10, years_average2 = 5, years_average3 = 1):
    #Gets the average Return of Investment Capital over the last 10, 5 and 1 year and does an average of that.
    try:
        roic1 = np.mean(df['ROIC'].tail(years_average1))
        roic2 = np.mean(df['ROIC'].tail(years_average2))
        roic3 = np.mean(df['ROIC'].tail(years_average3))
        roics = [roic1, roic2, roic3]
        roic = np.mean(roics)
    except KeyError:
        return None
    if math.isnan(roic):
        return None
    return round(float(roic),2)

In [40]:
def growth(df,column, years_growth = 10, years2 = 5, years3 = 2):
    #For the parameter of my choosing, it calculates the average growth over the last 10, 5 and 1 year and does an average of that.
    try:
        revenues = []
        if len(df) != 0:
            size = len(df[column].tail(years_growth))
        revenues.append(df[column].tail(years_growth))
        revenues.append(df[column].tail(years2))
        revenues.append(df[column].tail(years3))
        results = []
    except KeyError:
        if column == 'Revenue':
                return None, None
        else:
            return None
    for revenue in revenues:
        year = len(revenue)
        recent_value = float(revenue.tail(1))
        oldest_value = float(revenue.head(1))
        if math.isnan(recent_value) or math.isnan(oldest_value) or len(revenue) ==0:
            if column == 'Revenue':
                return None, None
            else:
                return None
        else:
            try:
                if recent_value == 0.0:
                    recent_value = 1.0
                if oldest_value == 0.0:
                    oldest_value = 1.0
                if recent_value > 0 and oldest_value > 0:
                    result = round(((recent_value/oldest_value)**(1/year)-1)*100,2)
                elif recent_value < 0 and oldest_value < 0:
                    recent_value = -recent_value
                    oldest_value = -oldest_value
                    result = -round(((recent_value/oldest_value)**(1/year)-1)*100,2)
                elif (recent_value < 0 and oldest_value > 0) or (recent_value > 0 and oldest_value < 0):
                    shift = 2*(abs(recent_value) + abs(oldest_value))
                    recent_value += shift
                    oldest_value += shift
                    result = round(((recent_value/oldest_value)**(1/year)-1)*100,2)
                results.append(result)
                
            except:
                print(recent_value)
                print(oldest_value)
            if column == 'Revenue':
                return round(np.mean(results),2), size
            else:
                return round(np.mean(results),2)
            
        

In [41]:
def get_Debt_repaiment(df, years_average = 3):
    #Calculates taking into account the Free cash flow of the company over the last 3 years, how many years it would take to pay all their 
    #Total Non-Current Liabilities.
    try:
        FCF = np.mean(df['Free Cash Flow'].tail(years_average))
    except KeyError:
        print('key_err fcf')
        return None
    try:
        Debt = float(df['Total Non-Current Liabilities'].tail(1))
    except KeyError:
        print('err debt')
        return None
    if math.isnan(Debt):
        Debt = 0
    if math.isnan(FCF) or FCF <0:
        FCF = 0.001
        
    return round(Debt/FCF,2)


In [42]:
def final(df,  debt_multiplier = 5, years_average = 5, roic_needed = 10, years_growth = 10):
    #For each stock, it calculates these metrics
    try:
        lst_boolean = []
        roic = get_ROIC(df)
        
        revenue_growth, size = growth(df,column='Revenue', years_growth = years_growth)
        net_income_growth = growth(df,column='Net Income', years_growth = years_growth)
        shares_outstasnding_growth = growth(df,column ='Weighted Avg. Shares Outs. Dil.', years_growth = years_growth)
        FCF_growth = growth(df,column='Free Cash Flow', years_growth = years_growth)
        debt_repaiment = get_Debt_repaiment(df, years_average)
        lst_with_numbers = [ roic, revenue_growth, net_income_growth, shares_outstasnding_growth, FCF_growth, debt_repaiment]#, FCF_price_ratio]

        if roic != None:
            if roic > roic_needed:
                lst_boolean.append(True)
            else:
                lst_boolean.append(False)
        else:
            lst_boolean.append(False)
        
        if revenue_growth != None:
            if revenue_growth > 0:
                lst_boolean.append(True)
            else:
                lst_boolean.append(False)
        else:
            lst_boolean.append(False)

        if net_income_growth != None:
            if net_income_growth > 0:
                lst_boolean.append(True)
            else:
                lst_boolean.append(False)
        else:
            lst_boolean.append(False)

        if shares_outstasnding_growth != None:
            if shares_outstasnding_growth < 0:
                lst_boolean.append(True)
            else:
                lst_boolean.append(False)
        else:
            lst_boolean.append(False)

        if FCF_growth != None:
            if FCF_growth > 0:
                lst_boolean.append(True)
            else:
                lst_boolean.append(False)
        else:
            lst_boolean.append(False)

        if debt_repaiment != None:
            if debt_repaiment < debt_multiplier:
                lst_boolean.append(True)
            else:
                lst_boolean.append(False)
        else:
            lst_boolean.append(False)

        return lst_with_numbers, lst_boolean, size
    except:
        print(roic)
        print(revenue_growth)
        print(net_income_growth)
        print(shares_outstasnding_growth)
        print(FCF_growth)
        print(debt_repaiment)

        print(df.head(10))

        
        
        
        
        
        return None, None

In [43]:
def do_everything():
    #For each stock in the folder, it calculates the above mentioned metrics
    lst = []
    path = os.getcwd()
    print(path)
    path = path + '/csvs'
    csv_files = glob.glob(os.path.join(path, "*.csv"))
    #print(len(csv_files))
    for i in csv_files:

        df = pd.read_csv( i, sep = ';', encoding = 'latin1')
        print('Stock:', i.split("\\")[-1][:-4])
        stock = i.split("\\")[-1][:-4]
        lst1,lst2, size = final( df, years_growth = 10)
        if isinstance(lst1, list):
            lst.append([lst1,lst2, stock, size])
        else:
            pass

    return lst

In [44]:
lol = do_everything()

c:\Users\Utilizador\Desktop\Python\Stocks
Stock: A
Stock: AA
Stock: AAC
Stock: AACG
Stock: AADI
Stock: AAIC
Stock: AAL
Stock: AAME
Stock: AAN
Stock: AAOI
Stock: AAON
Stock: AAP
Stock: AAPL
Stock: AAT
Stock: AATC
Stock: AAU
Stock: AAWW
Stock: AB
Stock: ABB
Stock: ABBV
Stock: ABC
Stock: ABCB
Stock: ABCL
Stock: ABCM
Stock: ABEO
Stock: ABEV
Stock: ABG
Stock: ABGI
Stock: ABIO
Stock: ABM
Stock: ABMC
Stock: ABMD
Stock: ABML
Stock: ABMT
Stock: ABNB
Stock: ABOS
Stock: ABQQ
Stock: ABR
Stock: ABSI
Stock: ABST
Stock: ABT
Stock: ABUS
Stock: ABVC
Stock: AC
Stock: ACA
Stock: ACAD
Stock: ACAHU
Stock: ACAN
Stock: ACAQ
Stock: ACB
Stock: ACBA
Stock: ACBI
Stock: ACBM
Stock: ACC
Stock: ACCD
Stock: ACCO
Stock: ACDI
Stock: ACEL
Stock: ACER
Stock: ACET
Stock: ACEV
Stock: ACEVU
Stock: ACFN
Stock: ACGL
Stock: ACH
Stock: ACHC
Stock: ACHL
Stock: ACHR
Stock: ACHV
Stock: ACI
Stock: ACII
Stock: ACIU
Stock: ACIW
Stock: ACKIT
Stock: ACKIU
Stock: ACLS
Stock: ACM
Stock: ACMR
Stock: ACN
Stock: ACNB
Stock: ACOR
Stock: ACP

In [45]:
#Calculates for each stock how many of the metrics the stock actually passes.
for i in lol:
    i[1] = np.array(i[1])
for i in lol:
    i.append(sum(i[1]))

In [46]:
#Select only those stocks that actually have all 6 metrics meet.
six_pillars = []
for i in lol:
    if i[4]==6:
        six_pillars.append(i)

In [47]:
def go_to_site(stock, site = 'https://roic.ai/company/'):
    url = site + stock 
    resp = http.request('GET', url)
    soup = BeautifulSoup(resp.data, 'html.parser')
    return soup

In [48]:
def get_table(soup, text, division = 'div', parameter = 'class' ):
    mydivs = soup.find_all(division, {parameter: text })
    lst = []
    for i in mydivs:
        lst.append(i.get_text())
    return lst

In [49]:
#For those stocks previously selected, it goes to the website and retrieves information about the marketcap, and calculates de PE and the FCF_ratio.

next_eigth_pillars = []
for i in six_pillars:
    n=0
    try:
        stock = i[2]
        site = go_to_site(stock = stock)
        table = get_table(site, text = 'block text-lg font-light', division = 'span')
        pe = float(table[0])
        market_cap = table[3]
        if market_cap[-1]== 'M':
            marketC = float(market_cap[:-1])
        elif market_cap[-1]== 'B':
            marketC = float(market_cap[:-1])*1000
        elif market_cap[-1]== 'T':
            marketC = float(market_cap[:-1])*1000000
        else:
            marketC = 100000000000000000000000000000000
        csv = pd.read_csv('csvs/' + stock + '.csv', encoding = 'latin1', sep = ';')
        FCF = float(csv['Free Cash Flow'].tail(1))
        FCF_ratio = round(marketC/FCF)
        if pe != None:
            if pe < 20:
                i[1] = np.append(i[1],True)
            else:
                i[1] = np.append(i[1],False)
        else:
            i[1] = np.append(i[1],False)
        if FCF_ratio != None:
            if FCF_ratio < 20:
                i[1] = np.append(i[1],True)
            else:
                i[1] = np.append(i[1],False)
        else:
            i[1] = np.append(i[1],False)
        i[0].append(pe)
        i[0].append(FCF_ratio)
        table1 = get_table(site, text = 'pl-1', division = 'div')
        business = table1[135]
        i[0].append(business)
        i[0].append(market_cap)
        n+=1
        next_eigth_pillars.append(i)
    except:
        pass




In [50]:
#Select only those stocks that actually have all 6 metrics meet.
for i in next_eigth_pillars:
    i[4] = sum(i[1])
    eight_pillars = []
for i in next_eigth_pillars:
    if i[4]==8:
        eight_pillars.append(i)

In [51]:
#Arranges the data so it can be turned into a Dataframe
index = []
columns = ['Number of pillars','ROIC', 'Revenue_growth', 'Net_income', 'Shares','FCF','Debt','PE_ratio', 'FCF_ratio','Business','Marketcap', 'ROIC_t', 'Revenue_growth_t', 'Net_income_t', 'Shares_t','FCF_t','Debt_t','PE_ratio_t','FCF_ratio_t', 'Min_size']
data = []
for i in eight_pillars:
    index.append(i[2])
    data.append(i[4])
    for j in i[0]:
        data.append(j)
    for j in i[1]:
        data.append(j)
    data.append(i[3])
data1 = np.reshape(data,((int(len(data)/len(columns)),len(columns))))
final = pd.DataFrame(data=data1, columns=columns, index=index)
eight = final[(final['Min_size']=='10')]

In [52]:
#Just removes unnecessary columns
eight = eight[[ 'PE_ratio','ROIC',	'Revenue_growth',	'Net_income',	'Shares',	'Debt',	'FCF_ratio', 'Business', 'Marketcap']]

In [53]:
#for each column, it gives each stock a number based on the sorted values for that column 
list_of_lists = []
for i in [ 'PE_ratio',	'ROIC',	'Revenue_growth',	'Net_income',	'Shares',	'Debt',	'FCF_ratio', 'Shares']:
    sorted = np.argsort(eight[i].astype(float), kind = 'stable')
    lst = [0]*len(sorted)
    for j in range(len(sorted)):
        lst[sorted[j]] = j
    #For these values, the higher they are, the better, so I just sort them in the opposite direction
    if i == 'ROIC' or i == 'Revenue_growth' or i == 'Net_income':
        ls = []
        for j in lst:
            ls.append(-j+(len(sorted)-1))
        lst = ls
    list_of_lists.append(lst)
sum_for_each_stock = []
for i in range(len(lst)):
    num = 0
    for j in range(len(list_of_lists)):
        num += list_of_lists[j][i]
    sum_for_each_stock.append(num)




In [54]:
eight['Summary'] = sum_for_each_stock
#The lower the Summary number, the better
eight.sort_values('Summary')

,PE_ratio,ROIC,Revenue_growth,Net_income,Shares,Debt,FCF_ratio,Business,Marketcap,Summary
LPX,3.16,39.7,10.25,47.11,-4.11,1.24,3,Building Products & Equipment,4.290B,51
ABG,5.49,59.44,7.8,20.56,-5.08,0.0,4,Auto & Truck Dealerships,3.924B,63
SWBI,3.28,39.77,9.9,31.74,-1.95,0.51,3,Aerospace & Defense,748.193M,109
SNBR,9.45,522.02,7.1,5.95,-6.86,2.58,3,"Furnishings, Fixtures & Appliances",773.451M,144
SIG,6.85,inf,6.99,7.89,-2.72,2.19,2,Luxury Goods,2.878B,148
PHM,4.97,14.56,11.19,25.18,-3.85,2.48,10,Residential Construction,9.598B,149
APAM,7.89,41.72,9.26,25.78,-1.63,1.46,6,Asset Management,2.546B,173
PLCE,3.95,20.96,0.57,11.49,-4.59,3.28,4,Apparel Retail,578.202M,181
HNNA,10.12,11.84,16.77,23.11,-2.48,0.88,8,Asset Management,77.869M,184
AMAT,13.02,23.96,10.22,49.02,-3.24,2.08,18,Semiconductor Equipment & Materials,84.698B,187
